# Data Clean

In [73]:
import pandas as pd

In [74]:
file=pd.read_csv('animals_comments.csv',index_col=None)
print(file.shape)

(5819470, 3)


In [10]:
file.isnull().sum()

creator_name    32050
userid              0
comment           488
dtype: int64

In [111]:
cat_saying=['cat','cats','bobcat','cheetah','cougar','jaguar','kitten','kitty','leopard','lion']
dog_saying=['dog','dogs','canine', 'pup', 'puppy', 'cur', 'doggy', 'hound', 'mongrel', 'mutt', 'pooch', 'stray', 'tyke', 'bowwow', 'domestic', 'fido', 'flea bag', 'tail-wagger']
saying=cat_saying+dog_saying+['pet','pets']
def myFunction(comment):
    try:
        if 'my' in comment and any(word in comment for word in saying):
            return 1
        else:
            return 0
    except:
        return 0

In [112]:
file['label'] = file.apply (lambda row: myFunction(row[2]),axis=1)

In [114]:
file.head(20)

,creator_name,userid,comment,label
0,Doug The Pug,87,I shared this to my friends and mom the were lol,0
1,Doug The Pug,87,Super cute 😀🐕🐶,0
2,bulletproof,530,stop saying get em youre literally dumb . have...,0
3,Meu Zoológico,670,Tenho uma jiboia e um largato,0
4,ojatro,1031,I wanna see what happened to the pigs after th...,0
5,Tingle Triggers,1212,Well shit now Im hungry,0
6,Hope For Paws - Official Rescue Channel,1806,when I saw the end it said to adopt I saw dif...,0
7,Hope For Paws - Official Rescue Channel,2036,Holy crap. That is quite literally the most ad...,0
8,Life Story,2637,武器はクエストで貰えるんじゃないんですか？,0
9,Brian Barczyk,2698,Call the teddy Larry,0


In [156]:
def get_rid_of_nulls(value):
    if pd.isnull(value):
        return 'null'
    else:
        return value

In [157]:
file['creator_name']=file['creator_name'].apply(get_rid_of_nulls)

In [196]:
file=file.dropna()

In [197]:
file.isnull().sum()

creator_name    0
userid          0
comment         0
dtype: int64

In [198]:
print(file.shape)

(5786942, 3)


In [199]:
file.head(5)

,creator_name,userid,comment
0,Doug The Pug,87,I shared this to my friends and mom the were lol
1,Doug The Pug,87,Super cute 😀🐕🐶
2,bulletproof,530,stop saying get em youre literally dumb . have...
3,Meu Zoológico,670,Tenho uma jiboia e um largato
4,ojatro,1031,I wanna see what happened to the pigs after th...


In [115]:
file.to_csv('clean_animals_comments.csv')

In [194]:
import csv
with open('clean_animals_comments.csv', 'rb') as f:
    reader = csv.reader(f)
    linenumber = 1
    try:
        for row in reader:
            linenumber += 1
    except Exception as e:
        print (("Error line %d: %s" % (linenumber, str(type(e)))))

Error line 1: <class '_csv.Error'>


In [ ]:
sample=pd.read_csv('clean_animals_comments.csv')
print(sample.type)

# ETL

In [1]:
import findspark
findspark.init()
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.types import *
sc =SparkContext()
sqlContext = SQLContext(sc)

In [2]:
df = sqlContext.read.load('animals_comments.csv', format='com.databricks.spark.csv', header='true', inferSchema='true')
df.show(10)

+--------------------+------+--------------------+
|        creator_name|userid|             comment|
+--------------------+------+--------------------+
|        Doug The Pug|  87.0|I shared this to ...|
|        Doug The Pug|  87.0|  Super cute  😀🐕🐶|
|         bulletproof| 530.0|stop saying get e...|
|       Meu Zoológico| 670.0|Tenho uma jiboia ...|
|              ojatro|1031.0|I wanna see what ...|
|     Tingle Triggers|1212.0|Well shit now Im ...|
|Hope For Paws - O...|1806.0|when I saw the en...|
|Hope For Paws - O...|2036.0|Holy crap. That i...|
|          Life Story|2637.0|武器はクエストで貰えるんじゃないん...|
|       Brian Barczyk|2698.0|Call the teddy Larry|
+--------------------+------+--------------------+
only showing top 10 rows



In [3]:
df.count()

5820035

In [4]:
df2 = df.dropna(thresh=3,subset=('creator_name','userid','comment'))
df2.show(5)

+-------------+------+--------------------+
| creator_name|userid|             comment|
+-------------+------+--------------------+
| Doug The Pug|  87.0|I shared this to ...|
| Doug The Pug|  87.0|  Super cute  😀🐕🐶|
|  bulletproof| 530.0|stop saying get e...|
|Meu Zoológico| 670.0|Tenho uma jiboia ...|
|       ojatro|1031.0|I wanna see what ...|
+-------------+------+--------------------+
only showing top 5 rows



In [5]:
df2.count()

5786944

In [8]:
from pyspark.sql.functions import udf,col

cat_saying=['cat','cats','bobcat','cheetah','cougar','jaguar','kitten','kitty','leopard','lion']
dog_saying=['dog','dogs','canine', 'pup', 'puppy', 'cur', 'doggy', 'hound', 'mongrel', 'mutt', 'pooch', 'stray', 'tyke', 'bowwow', 'domestic', 'fido', 'flea bag', 'tail-wagger']
saying=cat_saying+dog_saying+['pet','pets']
def myFunction(comment):
    try:
        if 'my' in comment and any(word in comment for word in saying):
            return True
        else:
            return False
    except:
        return False

func_udf = udf(myFunction, BooleanType())
df2 = df_clean.withColumn('dog_cat',func_udf(df_clean['text']))
df2.show()

Py4JJavaError: An error occurred while calling o52.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 42, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/macbookair/Platforms/spark/2.1.1/python/lib/pyspark.zip/pyspark/worker.py", line 125, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 3.5 than that in driver 3.6, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spark.api.python.PythonRunner$$anon$1.<init>(PythonRDD.scala:234)
	at org.apache.spark.api.python.PythonRunner.compute(PythonRDD.scala:152)
	at org.apache.spark.sql.execution.python.BatchEvalP

In [76]:
df_clean.show(5)

+-------------+------+--------------------+-------+--------------------+-----------------+
| creator_name|userid|             comment|dog_cat|                text|             name|
+-------------+------+--------------------+-------+--------------------+-----------------+
| Doug The Pug|  87.0|I shared this to ...|  false|[i, shared, this,...| [doug, the, pug]|
| Doug The Pug|  87.0|  Super cute  😀🐕🐶|  false|       [super, cute]| [doug, the, pug]|
|  bulletproof| 530.0|stop saying get e...|  false|[stop, saying, ge...|    [bulletproof]|
|Meu Zoológico| 670.0|Tenho uma jiboia ...|  false|[tenho, uma, jibo...|[meu, zool, gico]|
|       ojatro|1031.0|I wanna see what ...|  false|[i, wanna, see, w...|         [ojatro]|
+-------------+------+--------------------+-------+--------------------+-----------------+
only showing top 5 rows



In [9]:
cond = (df["comment"].like("%my dog%") | df["comment"].like("%I have a dog%")\
        | df["comment"].like("%my cat%") | df["comment"].like("%I have a cat%"))

df_clean = df.withColumn('dog_cat',cond)
df_clean.show()

+--------------------+------+--------------------+-------+
|        creator_name|userid|             comment|dog_cat|
+--------------------+------+--------------------+-------+
|        Doug The Pug|  87.0|I shared this to ...|  false|
|        Doug The Pug|  87.0|  Super cute  😀🐕🐶|  false|
|         bulletproof| 530.0|stop saying get e...|  false|
|       Meu Zoológico| 670.0|Tenho uma jiboia ...|  false|
|              ojatro|1031.0|I wanna see what ...|  false|
|     Tingle Triggers|1212.0|Well shit now Im ...|  false|
|Hope For Paws - O...|1806.0|when I saw the en...|  false|
|Hope For Paws - O...|2036.0|Holy crap. That i...|  false|
|          Life Story|2637.0|武器はクエストで貰えるんじゃないん...|  false|
|       Brian Barczyk|2698.0|Call the teddy Larry|  false|
|            The Dodo|2702.0|  😐🤔😓😢😭😭😭😭😟|  false|
|Hope For Paws - O...|2911.0|That mother cat l...|  false|
|Hope For Paws - O...|2911.0|Its people like H...|  false|
|   Talking Kitty Cat|2911.0|steve: No wet foo...|  false|
|    Brav

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer, RegexTokenizer, CountVectorizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [13]:
regexTokenizer = RegexTokenizer(inputCol="creator_name", outputCol="name", pattern="\\W")
df_clean = regexTokenizer.transform(df_clean)
df_clean.show(10)

+--------------------+------+--------------------+-------+--------------------+
|        creator_name|userid|             comment|dog_cat|                name|
+--------------------+------+--------------------+-------+--------------------+
|        Doug The Pug|  87.0|I shared this to ...|  false|    [doug, the, pug]|
|        Doug The Pug|  87.0|  Super cute  😀🐕🐶|  false|    [doug, the, pug]|
|         bulletproof| 530.0|stop saying get e...|  false|       [bulletproof]|
|       Meu Zoológico| 670.0|Tenho uma jiboia ...|  false|   [meu, zool, gico]|
|              ojatro|1031.0|I wanna see what ...|  false|            [ojatro]|
|     Tingle Triggers|1212.0|Well shit now Im ...|  false|  [tingle, triggers]|
|Hope For Paws - O...|1806.0|when I saw the en...|  false|[hope, for, paws,...|
|Hope For Paws - O...|2036.0|Holy crap. That i...|  false|[hope, for, paws,...|
|          Life Story|2637.0|武器はクエストで貰えるんじゃないん...|  false|       [life, story]|
|       Brian Barczyk|2698.0|Call the teddy

In [15]:
hashingTF = HashingTF(inputCol="name", outputCol="rawName", numFeatures=10000)
df_clean = hashingTF.transform(df_clean)
df_clean.show(10)

+--------------------+------+--------------------+-------+--------------------+--------------------+
|        creator_name|userid|             comment|dog_cat|                name|             rawName|
+--------------------+------+--------------------+-------+--------------------+--------------------+
|        Doug The Pug|  87.0|I shared this to ...|  false|    [doug, the, pug]|(10000,[5710,5885...|
|        Doug The Pug|  87.0|  Super cute  😀🐕🐶|  false|    [doug, the, pug]|(10000,[5710,5885...|
|         bulletproof| 530.0|stop saying get e...|  false|       [bulletproof]|(10000,[8647],[1.0])|
|       Meu Zoológico| 670.0|Tenho uma jiboia ...|  false|   [meu, zool, gico]|(10000,[2564,3283...|
|              ojatro|1031.0|I wanna see what ...|  false|            [ojatro]|(10000,[8067],[1.0])|
|     Tingle Triggers|1212.0|Well shit now Im ...|  false|  [tingle, triggers]|(10000,[4532,5257...|
|Hope For Paws - O...|1806.0|when I saw the en...|  false|[hope, for, paws,...|(10000,[1036,39

In [18]:
regexTokenizer = RegexTokenizer(inputCol="comment", outputCol="text", pattern="\\W")
df_clean = regexTokenizer.transform(df_clean)
df_clean.show(10)

+--------------------+------+--------------------+-------+--------------------+--------------------+--------------------+
|        creator_name|userid|             comment|dog_cat|                name|             rawName|                text|
+--------------------+------+--------------------+-------+--------------------+--------------------+--------------------+
|        Doug The Pug|  87.0|I shared this to ...|  false|    [doug, the, pug]|(10000,[5710,5885...|[i, shared, this,...|
|        Doug The Pug|  87.0|  Super cute  😀🐕🐶|  false|    [doug, the, pug]|(10000,[5710,5885...|       [super, cute]|
|         bulletproof| 530.0|stop saying get e...|  false|       [bulletproof]|(10000,[8647],[1.0])|[stop, saying, ge...|
|       Meu Zoológico| 670.0|Tenho uma jiboia ...|  false|   [meu, zool, gico]|(10000,[2564,3283...|[tenho, uma, jibo...|
|              ojatro|1031.0|I wanna see what ...|  false|            [ojatro]|(10000,[8067],[1.0])|[i, wanna, see, w...|
|     Tingle Triggers|1212.

In [19]:
hashingTF = HashingTF(inputCol="text", outputCol="rawComment", numFeatures=10000)
df_clean = hashingTF.transform(df_clean)
df_clean.show(10)

+--------------------+------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+
|        creator_name|userid|             comment|dog_cat|                name|             rawName|                text|          rawComment|
+--------------------+------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+
|        Doug The Pug|  87.0|I shared this to ...|  false|    [doug, the, pug]|(10000,[5710,5885...|[i, shared, this,...|(10000,[2293,4442...|
|        Doug The Pug|  87.0|  Super cute  😀🐕🐶|  false|    [doug, the, pug]|(10000,[5710,5885...|       [super, cute]|(10000,[349,2659]...|
|         bulletproof| 530.0|stop saying get e...|  false|       [bulletproof]|(10000,[8647],[1.0])|[stop, saying, ge...|(10000,[639,796,8...|
|       Meu Zoológico| 670.0|Tenho uma jiboia ...|  false|   [meu, zool, gico]|(10000,[2564,3283...|[tenho, uma, jibo...|(10000,[422,1878,...|
| 

In [27]:
(trainingData, testData) = df2.randomSplit([0.6, 0.4], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 3471658
Test Dataset Count: 2315286


In [28]:
cond = (df["comment"].like("%my dog%") | df["comment"].like("%I have a dog%")\
        | df["comment"].like("%my cat%") | df["comment"].like("%I have a cat%"))

trainingData_clean = trainingData.withColumn('dog_cat',cond)
trainingData_clean.show()

+--------------------+--------+--------------------+-------+
|        creator_name|  userid|             comment|dog_cat|
+--------------------+--------+--------------------+-------+
|#CameraLord™ • Ko...| 66527.0|Shout out from Ca...|  false|
|#CameraLord™ • Ko...| 70385.0|Billy just showed...|  false|
|#CameraLord™ • Ko...| 80582.0|             Snapped|  false|
|#CameraLord™ • Ko...| 96010.0|Billi Takin Over ...|  false|
|#CameraLord™ • Ko...| 96010.0|Subscribe for new...|  false|
|#CameraLord™ • Ko...| 96010.0|Thumbs Up If You ...|  false|
|#CameraLord™ • Ko...| 96010.0|Who rockin with t...|  false|
|#CameraLord™ • Ko...|144198.0|Lmfao oml folks d...|  false|
|#CameraLord™ • Ko...|179865.0|This the real ban...|  false|
|#CameraLord™ • Ko...|185783.0|         okay okay!!|  false|
|#CameraLord™ • Ko...|252737.0|      this goes hard|  false|
|#CameraLord™ • Ko...|254431.0|Billy a beast... ...|  false|
|#CameraLord™ • Ko...|254431.0|How u Wuga World ...|  false|
|#CameraLord™ • Ko...|26

In [29]:
regexTokenizer_name = RegexTokenizer(inputCol="creator_name", outputCol="name", pattern="\\W")
hashingTF_name = HashingTF(inputCol="name", outputCol="rawName", numFeatures=10000)
regexTokenizer_comment = RegexTokenizer(inputCol="comment", outputCol="text", pattern="\\W")
hashingTF_comment = HashingTF(inputCol="text", outputCol="rawComment", numFeatures=10000)
pipeline = Pipeline(stages=[regexTokenizer_name,hashingTF_name,regexTokenizer_comment,hashingTF_comment])

In [31]:
pipelineFit = pipeline.fit(trainingData_clean)
trainingDataset = pipelineFit.transform(trainingData_clean)
trainingDataset.show(5)

+--------------------+-------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+
|        creator_name| userid|             comment|dog_cat|                name|             rawName|                text|          rawComment|
+--------------------+-------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+
|#CameraLord™ • Ko...|66527.0|Shout out from Ca...|  false|[cameralord, kor,...|(10000,[3203,7884...|[shout, out, from...|(10000,[128,2495,...|
|#CameraLord™ • Ko...|70385.0|Billy just showed...|  false|[cameralord, kor,...|(10000,[3203,7884...|[billy, just, sho...|(10000,[307,368,1...|
|#CameraLord™ • Ko...|80582.0|             Snapped|  false|[cameralord, kor,...|(10000,[3203,7884...|           [snapped]|(10000,[3350],[1.0])|
|#CameraLord™ • Ko...|96010.0|Billi Takin Over ...|  false|[cameralord, kor,...|(10000,[3203,7884...|[billi, takin, ov...|(10000,[1564,4

In [33]:
cond = (df["comment"].like("%my dog%") | df["comment"].like("%I have a dog%")\
        | df["comment"].like("%my cat%") | df["comment"].like("%I have a cat%"))

testData_clean = testData.withColumn('dog_cat',cond)
testData_clean.show(5)

+--------------------+-------+--------------------+-------+
|        creator_name| userid|             comment|dog_cat|
+--------------------+-------+--------------------+-------+
|#CameraLord™ • Ko...|41444.0|                💯💯|  false|
|#CameraLord™ • Ko...|93552.0|BOONK GANG🤘🤘🤘?...|  false|
|#CameraLord™ • Ko...|94581.0|I really like thi...|  false|
|#CameraLord™ • Ko...|96010.0|Billi slay him or...|  false|
|#CameraLord™ • Ko...|96010.0|Billy and Swagg t...|  false|
+--------------------+-------+--------------------+-------+
only showing top 5 rows



In [76]:
pipelineFit = pipeline.fit(testData_clean)
testDataset = pipelineFit.transform(testData_clean)
testDataset.show(5)

+--------------------+-------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+
|        creator_name| userid|             comment|dog_cat|                name|             rawName|                text|          rawComment|
+--------------------+-------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+
|#CameraLord™ • Ko...|41444.0|                💯💯|  false|[cameralord, kor,...|(10000,[3203,7884...|                  []|       (10000,[],[])|
|#CameraLord™ • Ko...|93552.0|BOONK GANG🤘🤘🤘?...|  false|[cameralord, kor,...|(10000,[3203,7884...|       [boonk, gang]|(10000,[1883,8535...|
|#CameraLord™ • Ko...|94581.0|I really like thi...|  false|[cameralord, kor,...|(10000,[3203,7884...|[i, really, like,...|(10000,[1310,3330...|
|#CameraLord™ • Ko...|96010.0|Billi slay him or...|  false|[cameralord, kor,...|(10000,[3203,7884...|[billi, slay, him...|(10000,[1187,4491..

In [43]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['rawName', 'rawComment'],outputCol='features')

training = assembler.transform(trainingDataset)
training.show(5)

+--------------------+-------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        creator_name| userid|             comment|dog_cat|                name|             rawName|                text|          rawComment|            features|
+--------------------+-------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+
|#CameraLord™ • Ko...|66527.0|Shout out from Ca...|  false|[cameralord, kor,...|(10000,[3203,7884...|[shout, out, from...|(10000,[128,2495,...|(20000,[3203,7884...|
|#CameraLord™ • Ko...|70385.0|Billy just showed...|  false|[cameralord, kor,...|(10000,[3203,7884...|[billy, just, sho...|(10000,[307,368,1...|(20000,[3203,7884...|
|#CameraLord™ • Ko...|80582.0|             Snapped|  false|[cameralord, kor,...|(10000,[3203,7884...|           [snapped]|(10000,[3350],[1.0])|(20000,[3203,7884...|
|#CameraLo

In [66]:
from pyspark.sql.types import IntegerType
train=training.withColumn("label", training['dog_cat'].cast(IntegerType()))
train.show(5)

+--------------------+-------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|        creator_name| userid|             comment|dog_cat|                name|             rawName|                text|          rawComment|            features|label|
+--------------------+-------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|#CameraLord™ • Ko...|66527.0|Shout out from Ca...|  false|[cameralord, kor,...|(10000,[3203,7884...|[shout, out, from...|(10000,[128,2495,...|(20000,[3203,7884...|    0|
|#CameraLord™ • Ko...|70385.0|Billy just showed...|  false|[cameralord, kor,...|(10000,[3203,7884...|[billy, just, sho...|(10000,[307,368,1...|(20000,[3203,7884...|    0|
|#CameraLord™ • Ko...|80582.0|             Snapped|  false|[cameralord, kor,...|(10000,[3203,7884...|           [snapped]|(10000,[3350],[1.0])|(2

In [77]:
test = assembler.transform(testDataset)
test.show(5)

+--------------------+-------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        creator_name| userid|             comment|dog_cat|                name|             rawName|                text|          rawComment|            features|
+--------------------+-------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+
|#CameraLord™ • Ko...|41444.0|                💯💯|  false|[cameralord, kor,...|(10000,[3203,7884...|                  []|       (10000,[],[])|(20000,[3203,7884...|
|#CameraLord™ • Ko...|93552.0|BOONK GANG🤘🤘🤘?...|  false|[cameralord, kor,...|(10000,[3203,7884...|       [boonk, gang]|(10000,[1883,8535...|(20000,[3203,7884...|
|#CameraLord™ • Ko...|94581.0|I really like thi...|  false|[cameralord, kor,...|(10000,[3203,7884...|[i, really, like,...|(10000,[1310,3330...|(20000,[3203,7884...|
|#CameraLord™ •

In [78]:
testing=test.withColumn("label", test['dog_cat'].cast(IntegerType()))
testing.show(5)

+--------------------+-------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|        creator_name| userid|             comment|dog_cat|                name|             rawName|                text|          rawComment|            features|label|
+--------------------+-------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|#CameraLord™ • Ko...|41444.0|                💯💯|  false|[cameralord, kor,...|(10000,[3203,7884...|                  []|       (10000,[],[])|(20000,[3203,7884...|    0|
|#CameraLord™ • Ko...|93552.0|BOONK GANG🤘🤘🤘?...|  false|[cameralord, kor,...|(10000,[3203,7884...|       [boonk, gang]|(10000,[1883,8535...|(20000,[3203,7884...|    0|
|#CameraLord™ • Ko...|94581.0|I really like thi...|  false|[cameralord, kor,...|(10000,[3203,7884...|[i, really, like,...|(10000,[1310,3330...|(20000,

In [81]:
lr = LogisticRegression(featuresCol='features', labelCol='label',maxIter=10, regParam=0.01, elasticNetParam=0.8)
lrModel = lr.fit(train)

In [82]:
prediction = lrModel.transform(testing)
prediction.show(5)

+--------------------+-------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|        creator_name| userid|             comment|dog_cat|                name|             rawName|                text|          rawComment|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|#CameraLord™ • Ko...|41444.0|                💯💯|  false|[cameralord, kor,...|(10000,[3203,7884...|                  []|       (10000,[],[])|(20000,[3203,7884...|    0|[5.34396707301585...|[0.99524582729688...|       0.0|
|#CameraLord™ • Ko...|93552.0|BOONK GANG🤘🤘🤘?...|  false|[cameralord, kor,...|(10000,[3203,7884...|       [

In [83]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(prediction)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(prediction, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(prediction, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(prediction, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 0.081
MSE: 0.007
MAE: 0.007
r2: -0.027
